In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv('telescope_data.csv',index_col=0)
data.head()

In [ ]:
cols = data.select_dtypes(exclude='object').columns
from sklearn.preprocessing import OrdinalEncoder
myimpute = OrdinalEncoder(handle_unknown='error')
data['class'] = myimpute.fit_transform(data[['class']])

In [ ]:
data.head()

In [ ]:
import seaborn as sns
corr = data.corr()
sns.heatmap(data=corr,annot=True)
plt.show()

In [ ]:
for col in cols:
    plt.hist(data[data['class']==1][col],color='blue',density=True,label='Hadron',alpha=0.7)
    plt.hist(data[data['class']==0][col],color='red',density=True,label='Gamma',alpha=0.7)
    # plt.hist(data=data,x=col,bins=10)
    plt.title(col)
    plt.xlabel(col)
    plt.ylabel('class')
    plt.legend()
    plt.show()

In [ ]:
train,valid,test = np.split(data.sample(frac=1),[int(0.6*len(data)),int(0.8*len(data))])

In [ ]:
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import StandardScaler

In [ ]:
def scale_dataset(dataframe,oversample=False):
    X = dataframe[dataframe.columns[:-1]]
    y = dataframe[dataframe.columns[-1]]

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    if oversample:
        ros = RandomOverSampler()
        X,y = ros.fit_resample(X,y)

    data = np.hstack((X,np.reshape(y,(-1,1))))
    return data,X,y

In [ ]:
print(len(train[train['class']==1]))
print(len(train[train['class']==0]))

In [ ]:
data.head()

In [ ]:
train,X_train,y_train = scale_dataset(train,oversample=True)

In [ ]:
sum(y_train == 0)

In [ ]:
sum(y_train == 1)

In [ ]:
valid,X_valid,y_valid = scale_dataset(valid,oversample=False)
test,X_test,y_test = scale_dataset(test,oversample=False)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn_model = KNeighborsClassifier(n_neighbors=3)
knn_model.fit(X_train,y_train)
pred = knn_model.predict(X_valid)
pred

In [ ]:
y_valid

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_pred=pred,y_true=y_valid))

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
nb_model = GaussianNB()
nb_model = nb_model.fit(X_train,y_train)

In [ ]:
y_pred = nb_model.predict(X_valid)

In [ ]:
print(classification_report(y_pred=y_pred,y_true=y_valid))

In [ ]:
from sklearn.linear_model import LogisticRegression
lr_model = LogisticRegression()
lr_model = lr_model.fit(X_train,y_train)

In [ ]:
y_pred = lr_model.predict(X_valid)
print(classification_report(y_pred=y_pred,y_true=y_valid))

In [ ]:
from sklearn.svm import SVC
sv_model = SVC()
sv_model = sv_model.fit(X_train,y_train)

In [ ]:
y_pred = sv_model.predict(X_valid)

In [ ]:
print(classification_report(y_pred=y_pred,y_true=y_valid))

In [ ]:
import tensorflow as tf

In [ ]:
nn_model = tf.keras.Sequential([
    tf.keras.layers.Dense(32,activation='relu',input_shape=(10,)),
    tf.keras.layers.Dense(32,activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
])

In [ ]:
nn_model.compile(optimizer=tf.keras.optimizers.Adam(0.001),loss='binary_crossentropy',metrics=['accracy'])
#1:53:06